In [1]:
import numpy as np
import random
import itertools
import scipy.misc
import matplotlib.pyplot as plt
import pandas as pd



In [59]:
nstates = 8
nactions = 2


# build T and R for a simple 2 stage choice task

Tsas = np.zeros([nstates,nactions,nstates]) 

avail_actions = np.array([[0,1],[2,3],[4,5],[6],[7],[8],[9],[]])
next_states = np.array([1,2,3,4,5,6,7,7,7,7])
n_total_actions = 10

T_2d = np.zeros([n_total_actions, nstates])
for i in np.arange(n_total_actions):
    T_2d[i,next_states[i]] = 1
    
for s in np.arange(nstates):
    this_state_options = avail_actions[s]
    this_state_probs = T_2d[this_state_options,:]
    Tsas[s,0:np.size(this_state_options),:] = this_state_probs
    

# number of available actions per state
nActions = np.array([2,2,2,1,1,1,0])

Rs = np.array([0,0,0,-10, 5, 3, 2])
Rsas sas = np.zeros(nstates,nactions,nstates)

SyntaxError: invalid syntax (<ipython-input-59-b1e34e3dcf62>, line 27)